In [1]:
import numpy as np
import nibabel as nib
import pandas as pd
from pathlib import Path
import tqdm
from tqdm import tqdm

In [2]:
ROOT = "C:\\Users\\kiree\\Documents\\REC\\SEM-8\\Proj\\Benchmarking\\Method 2\\Task06_Lung\\"
TRAIN_DATA_PATH = ROOT + "imagesTr/"
TRAIN_LABEL_PATH = ROOT + "labelsTr/"

In [3]:
df = pd.DataFrame(columns=['data','label'])

In [4]:
import os

augment = ['normal', 'rotate_l', 'rotate_r', 'gaussian_blur']
for root, dirs, files in os.walk(TRAIN_DATA_PATH):
    for file in files:
        if not file.startswith("._"):
            temp = pd.DataFrame({ 'data' : [TRAIN_DATA_PATH + file], 'label' : [TRAIN_LABEL_PATH + file] })
            df = pd.concat([df, temp])

df

,data,label
0,C:\Users\kiree\Documents\REC\SEM-8\Proj\Benchm...,C:\Users\kiree\Documents\REC\SEM-8\Proj\Benchm...
0,C:\Users\kiree\Documents\REC\SEM-8\Proj\Benchm...,C:\Users\kiree\Documents\REC\SEM-8\Proj\Benchm...
0,C:\Users\kiree\Documents\REC\SEM-8\Proj\Benchm...,C:\Users\kiree\Documents\REC\SEM-8\Proj\Benchm...
0,C:\Users\kiree\Documents\REC\SEM-8\Proj\Benchm...,C:\Users\kiree\Documents\REC\SEM-8\Proj\Benchm...
0,C:\Users\kiree\Documents\REC\SEM-8\Proj\Benchm...,C:\Users\kiree\Documents\REC\SEM-8\Proj\Benchm...
...,...,...
0,C:\Users\kiree\Documents\REC\SEM-8\Proj\Benchm...,C:\Users\kiree\Documents\REC\SEM-8\Proj\Benchm...
0,C:\Users\kiree\Documents\REC\SEM-8\Proj\Benchm...,C:\Users\kiree\Documents\REC\SEM-8\Proj\Benchm...
0,C:\Users\kiree\Documents\REC\SEM-8\Proj\Benchm...,C:\Users\kiree\Documents\REC\SEM-8\Proj\Benchm...
0,C:\Users\kiree\Documents\REC\SEM-8\Proj\Benchm...,C:\Users\kiree\Documents\REC\SEM-8\Proj\Benchm...


In [5]:
df = df.sort_values(by=['data'])
df

,data,label
0,C:\Users\kiree\Documents\REC\SEM-8\Proj\Benchm...,C:\Users\kiree\Documents\REC\SEM-8\Proj\Benchm...
0,C:\Users\kiree\Documents\REC\SEM-8\Proj\Benchm...,C:\Users\kiree\Documents\REC\SEM-8\Proj\Benchm...
0,C:\Users\kiree\Documents\REC\SEM-8\Proj\Benchm...,C:\Users\kiree\Documents\REC\SEM-8\Proj\Benchm...
0,C:\Users\kiree\Documents\REC\SEM-8\Proj\Benchm...,C:\Users\kiree\Documents\REC\SEM-8\Proj\Benchm...
0,C:\Users\kiree\Documents\REC\SEM-8\Proj\Benchm...,C:\Users\kiree\Documents\REC\SEM-8\Proj\Benchm...
...,...,...
0,C:\Users\kiree\Documents\REC\SEM-8\Proj\Benchm...,C:\Users\kiree\Documents\REC\SEM-8\Proj\Benchm...
0,C:\Users\kiree\Documents\REC\SEM-8\Proj\Benchm...,C:\Users\kiree\Documents\REC\SEM-8\Proj\Benchm...
0,C:\Users\kiree\Documents\REC\SEM-8\Proj\Benchm...,C:\Users\kiree\Documents\REC\SEM-8\Proj\Benchm...
0,C:\Users\kiree\Documents\REC\SEM-8\Proj\Benchm...,C:\Users\kiree\Documents\REC\SEM-8\Proj\Benchm...


In [6]:
import cv2

In [7]:
save_root = Path(ROOT + "Preprocessed")
counter = 0
mod = 0
for i in tqdm(range(len(df))):
    ct_data = nib.load(df.iloc[i, 0]).get_fdata()
    label_data = nib.load(df.iloc[i, 1]).get_fdata()

    start_pos = 0
    end_pos = 0
    m2_start_pos = 0
    m2_end_pos = 0
    m1_start_pos = 0
    m1_end_pos = 0
 
    for j in range(label_data.shape[-1]):
        if label_data[:, :, j].sum() > 0 and m1_start_pos == 0:
            m1_start_pos = j
        
        if m1_start_pos > 0 and label_data[:, :, j].sum() == 0 and m1_end_pos == 0:
            m1_end_pos = j-1

    for j in range(label_data.shape[-1]-1, 0, -1):
        if label_data[:, :, j].sum() > 0 and m2_end_pos == 0:
            m2_end_pos = j
        
        if m2_end_pos > 0 and label_data[:, :, j].sum() == 0 and m2_start_pos == 0:
            m2_start_pos = j-1

    if (m1_end_pos - m1_start_pos) >= (m2_end_pos - m2_start_pos):
        start_pos = m1_start_pos
        end_pos = m1_end_pos
    else:
        start_pos = m2_start_pos
        end_pos = m2_end_pos

    ct_data = ct_data[:, :, start_pos-22:end_pos+20]
    label_data = label_data[:, :, start_pos-22:end_pos+20]

    if i < 50:
        current_data_path = save_root/"train"/"data"
        current_label_path = save_root/"train"/"label"
    else:
        current_data_path = save_root/"test"/"data"
        current_label_path = save_root/"test"/"label"

    for j in range(ct_data.shape[-1]):
        slice = ct_data[:,:,j]
        mask = label_data[:,:,j]

        slice = cv2.resize(slice,(256, 256))
        mask = cv2.resize(mask, (256, 256))

        slice = (slice - slice.min()) / (slice.max() - slice.min())
        mask = (mask - mask.min()) / (mask.max() - mask.min()) if mask.max() != 0 else mask
        slice = np.expand_dims(slice, 0)
        mask = np.expand_dims(mask, 0)
        slice = np.expand_dims(slice, 0)
        mask = np.expand_dims(mask, 0)
        
        current_data_path.mkdir(parents=True, exist_ok=True)
        current_label_path.mkdir(parents=True, exist_ok=True)
        
        np.save(current_data_path/str(counter), slice)
        np.save(current_label_path/str(counter), mask)

        counter += 1


100%|██████████| 63/63 [01:57<00:00,  1.87s/it]


In [8]:
# save_root = Path(ROOT + "Preprocessed")
# total_slice_count = 0
# counter = 0
# mod = 0
# for i in tqdm(range(len(df))):
#     ct_data = nib.load(df.iloc[i, 0]).get_fdata()
#     label_data = nib.load(df.iloc[i, 1]).get_fdata()
     
#     ct_data = ct_data[:, :, 30:]
#     label_data = label_data[:, :, 30:]

#     total_slice_count += ct_data.shape[-1]

#     if i < 57:
#         current_data_path = save_root/"train"/"data"
#         current_label_path = save_root/"train"/"label"
#     else:
#         current_data_path = save_root/"test"/"data"
#         current_label_path = save_root/"test"/"label"
    
#     for _ in range(ct_data.shape[-1]):
#         slice = ct_data[:,:,_]
#         mask = label_data[:,:,_]

#         slice = cv2.resize(slice,(256, 256))
#         mask = cv2.resize(slice, (256, 256))

#         slice = (slice - slice.min()) / (slice.max() - slice.min())
#         mask = (mask - mask.min()) / (mask.max() - mask.min())
#         slice = np.expand_dims(slice, 0)
#         mask = np.expand_dims(mask, 0)
#         slice = np.expand_dims(slice, 0)
#         mask = np.expand_dims(mask, 0)
        
#         current_data_path.mkdir(parents=True, exist_ok=True)
#         current_label_path.mkdir(parents=True, exist_ok=True)
        
#         np.save(current_data_path/str(counter), slice)
#         np.save(current_label_path/str(counter), mask)

#         counter += 1

# print(total_slice_count)

In [9]:
# path = Path("Task06_Lung/Preprocessed/train/2")  # Select a subject. Check the folder if it exists
# list(path.glob("*"))

In [10]:
# ROOT = 'Task06_Lung/'

In [11]:
# TEST_DATA_PATH = 'Task06_Lung/Preprocessed/train/data/'
# TEST_LABEL_PATH = 'Task06_Lung/Preprocessed/train/label/'

In [12]:
# fig = plt.figure()
# camera = Camera(fig)

# for _, __, files in os.walk(TEST_DATA_PATH):
#     for file in files:
#         print(file)
#         # ct = np.load(TEST_DATA_PATH + file)
#         mask = np.load(TEST_LABEL_PATH + file)

#         # plt.imshow(ct[0, 0, :, :], cmap="bone")
#         # mask_ = np.ma.masked_where(mask[0, 0, :, :]==0, mask[0, 0, :, :])
#         plt.imshow(mask[0, 0, :, :], alpha=0.5)
#         plt.axis("off")
#         camera.snap()

In [13]:
# animation = camera.animate()
# HTML(animation.to_html5_video())

In [14]:
# import matplotlib
# matplotlib.rcParams['animation.embed_limit'] = 2**128